## Notebook to convert audio files from the ESC-50 dataset into spectrograms using a variety of transforms

### Requires librosa for linear STFT / mel STFT / CQT and pyWavelets for CWT

In [1]:
%matplotlib inline
## kernel: python 3.5
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa # https://github.com/librosa/librosa
import librosa.display
import re
import warnings

import tiffspect


# Set some project parameters
K_SR = 22050 #sampling rate
K_FFTSIZE = 1024 # also used for window length where that parameter is called for
K_HOP = 512 #fft hop length
K_DUR = 5.0 # make all files this duration (sec)

main_dir = "./"

# location of subdirectories of ogg files organized by category
K_OGGDIR = main_dir + "ESC-50"
# location to write the wav files (converted from ogg)
K_WAVEDIR = main_dir + "ESC-50-wav"

In [2]:
def get_subdirs(a_dir):
    """ Returns a list of sub directory names in a_dir """ 
    return [name for name in os.listdir(a_dir)
            if (os.path.isdir(os.path.join(a_dir, name)) and not (name.startswith('.')))]

def listDirectory(directory, fileExtList):                                        
    """Returns a list of file info objects in directory that extension in the list fileExtList - include the . in your extension string"""
    fnameList = [os.path.normcase(f)
                for f in os.listdir(directory)
                    if (not(f.startswith('.')))]            
    fileList = [os.path.join(directory, f) 
               for f in fnameList
                if os.path.splitext(f)[1] in fileExtList]  
    return fileList , fnameList

def dirs2labelfile(parentdir, labelfile):
    """takes subdirectories of parentdir and writes them, one per line, to labelfile"""
    namelist = get_subdirs(parentdir)
    with open(labelfile, mode='wt', encoding='utf-8') as myfile:
    #with open(labelfile, mode='wt') as myfile:
        myfile.write('\n'.join(namelist))

### Convert and standardize the .ogg files into .wav files

In [3]:
# Use this for .ogg to .wav conversion. Clip/pad to same length.

def esc50Ogg2Wav (topdir, outdir, dur, srate) :
    """ 
        Creates regularlized wave files for the ogg files in the ESC-50 dataset. 
        Creates class folders for the wav files in outdir with the same structure found in topdir.
        
        Parameters
            topdir - the ESC-50 dir containing class folders. 
            outdir - the top level directory to write wave files to (written in to class subfolders)
            dur - (in seconds) all files will be truncated or zeropadded to have this duration given the srate
            srate - input files will be resampled to srate as they are read in before being saved as wav files
    """ 
    sample_length = int(dur * srate)
    try:
        os.stat(outdir)  # test for existence
    except:
        os.mkdir(outdir) # create if necessary
        
    subdirs = get_subdirs(topdir)
    for subdir in subdirs :

        fullpaths, _ = listDirectory(topdir + '/' + subdir, '.ogg')

        for idx in range(len(fullpaths)) : 
            fname = os.path.basename(fullpaths[idx])
            # librosa.load resamples to sr, clips to duration, combines channels. 
            audiodata, samplerate = librosa.load(fullpaths[idx], sr=srate, mono=True, duration=dur) # resamples if necessary (some esc-50 files are in 48K)
            # just checking ..... 
            if (samplerate != srate) :
                print('You got a sound file ' + subdir  +  '/' +  fname + ' with sample rate ' + str(samplerate) + '!')
                print(' ********* BAD SAMPLE RATE ******** ')
            if (audiodata.ndim != 1) :
                print('You got a sound file ' + subdir  +  '/' +  fname + ' with ' + str(audiodata.ndim) + ' channels!')
                audiodata = stereo2mono(audiodata)
            if (len(audiodata) > sample_length) :
                print('You got a long sound file ' + subdir  +  '/' +  fname + ' with shape ' + str(audiodata.shape) + '!')
                audiodata = np.resize(audiodata, sample_length)
                # print('  ..... and len(audiodata) = ' + str(len(audiodata)) + ', while sample_length is sposed to be ' + str(sample_length))
                print('trimming data to shape ' + str(audiodata.shape))
            if (len(audiodata) < sample_length) :
                print('You got a short sound file ' + subdir  +  '/' +  fname + ' with shape ' + str(audiodata.shape) + '!')
                audiodata = np.concatenate([audiodata, np.zeros((sample_length-len(audiodata)))])
                print('zero padding data to shape ' + str(audiodata.shape))
            
            # write the file out as a wave file
            try:
                os.stat(outdir + '/'  + subdir) # test for existence
            except:
                os.mkdir(outdir + '/' + subdir) # create if necessary
            print('creating ' + outdir + '/'  + subdir)
            librosa.output.write_wav(outdir + '/' + subdir + '/' + os.path.splitext(fname)[0] + '.wav', audiodata, samplerate)
    print("COMPLETE")    

In [4]:
esc50Ogg2Wav(K_OGGDIR, K_WAVEDIR, K_DUR, K_SR)

creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating ./ESC-50-wav/103 - Pig
creating

creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/403 - Keyboard typing
creating ./ESC-50-wav/109 - Shee

creating ./ESC-50-wav/507 - Church bells
creating ./ESC-50-wav/507 - Church bells
creating ./ESC-50-wav/507 - Church bells
creating ./ESC-50-wav/507 - Church bells
creating ./ESC-50-wav/507 - Church bells
creating ./ESC-50-wav/507 - Church bells
creating ./ESC-50-wav/507 - Church bells
creating ./ESC-50-wav/507 - Church bells
creating ./ESC-50-wav/507 - Church bells
creating ./ESC-50-wav/507 - Church bells
creating ./ESC-50-wav/507 - Church bells
creating ./ESC-50-wav/507 - Church bells
creating ./ESC-50-wav/507 - Church bells
creating ./ESC-50-wav/507 - Church bells
creating ./ESC-50-wav/507 - Church bells
creating ./ESC-50-wav/104 - Cow
creating ./ESC-50-wav/104 - Cow
creating ./ESC-50-wav/104 - Cow
creating ./ESC-50-wav/104 - Cow
creating ./ESC-50-wav/104 - Cow
creating ./ESC-50-wav/104 - Cow
creating ./ESC-50-wav/104 - Cow
creating ./ESC-50-wav/104 - Cow
creating ./ESC-50-wav/104 - Cow
creating ./ESC-50-wav/104 - Cow
creating ./ESC-50-wav/104 - Cow
creating ./ESC-50-wav/104 - Cow
c

creating ./ESC-50-wav/203 - Crackling fire
creating ./ESC-50-wav/203 - Crackling fire
creating ./ESC-50-wav/203 - Crackling fire
creating ./ESC-50-wav/203 - Crackling fire
creating ./ESC-50-wav/203 - Crackling fire
creating ./ESC-50-wav/203 - Crackling fire
creating ./ESC-50-wav/203 - Crackling fire
creating ./ESC-50-wav/203 - Crackling fire
creating ./ESC-50-wav/203 - Crackling fire
You got a short sound file 203 - Crackling fire/4-164661-B.ogg with shape (80384,)!
zero padding data to shape (110250,)
creating ./ESC-50-wav/203 - Crackling fire
creating ./ESC-50-wav/203 - Crackling fire
creating ./ESC-50-wav/203 - Crackling fire
creating ./ESC-50-wav/203 - Crackling fire
creating ./ESC-50-wav/203 - Crackling fire
creating ./ESC-50-wav/203 - Crackling fire
creating ./ESC-50-wav/203 - Crackling fire
creating ./ESC-50-wav/203 - Crackling fire
creating ./ESC-50-wav/203 - Crackling fire
creating ./ESC-50-wav/203 - Crackling fire
creating ./ESC-50-wav/203 - Crackling fire
creating ./ESC-50-w

creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./ESC-50-wav/509 - Fireworks
creating ./E

creating ./ESC-50-wav/204 - Crickets
creating ./ESC-50-wav/204 - Crickets
creating ./ESC-50-wav/204 - Crickets
creating ./ESC-50-wav/204 - Crickets
creating ./ESC-50-wav/204 - Crickets
creating ./ESC-50-wav/204 - Crickets
creating ./ESC-50-wav/204 - Crickets
creating ./ESC-50-wav/204 - Crickets
creating ./ESC-50-wav/204 - Crickets
creating ./ESC-50-wav/204 - Crickets
creating ./ESC-50-wav/204 - Crickets
creating ./ESC-50-wav/204 - Crickets
creating ./ESC-50-wav/208 - Pouring water
creating ./ESC-50-wav/208 - Pouring water
creating ./ESC-50-wav/208 - Pouring water
creating ./ESC-50-wav/208 - Pouring water
creating ./ESC-50-wav/208 - Pouring water
creating ./ESC-50-wav/208 - Pouring water
creating ./ESC-50-wav/208 - Pouring water
creating ./ESC-50-wav/208 - Pouring water
creating ./ESC-50-wav/208 - Pouring water
creating ./ESC-50-wav/208 - Pouring water
creating ./ESC-50-wav/208 - Pouring water
creating ./ESC-50-wav/208 - Pouring water
creating ./ESC-50-wav/208 - Pouring water
creating .

creating ./ESC-50-wav/404 - Door - wood creaks
creating ./ESC-50-wav/404 - Door - wood creaks
creating ./ESC-50-wav/404 - Door - wood creaks
creating ./ESC-50-wav/404 - Door - wood creaks
creating ./ESC-50-wav/404 - Door - wood creaks
creating ./ESC-50-wav/404 - Door - wood creaks
creating ./ESC-50-wav/404 - Door - wood creaks
creating ./ESC-50-wav/404 - Door - wood creaks
creating ./ESC-50-wav/404 - Door - wood creaks
creating ./ESC-50-wav/404 - Door - wood creaks
creating ./ESC-50-wav/404 - Door - wood creaks
creating ./ESC-50-wav/404 - Door - wood creaks
creating ./ESC-50-wav/404 - Door - wood creaks
creating ./ESC-50-wav/404 - Door - wood creaks
creating ./ESC-50-wav/404 - Door - wood creaks
creating ./ESC-50-wav/404 - Door - wood creaks
creating ./ESC-50-wav/404 - Door - wood creaks
creating ./ESC-50-wav/404 - Door - wood creaks
creating ./ESC-50-wav/404 - Door - wood creaks
creating ./ESC-50-wav/404 - Door - wood creaks
creating ./ESC-50-wav/404 - Door - wood creaks
creating ./ES

creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening
creating ./ESC-50-wav/405 - Can opening


creating ./ESC-50-wav/303 - Clapping
creating ./ESC-50-wav/303 - Clapping
creating ./ESC-50-wav/303 - Clapping
creating ./ESC-50-wav/303 - Clapping
creating ./ESC-50-wav/303 - Clapping
creating ./ESC-50-wav/303 - Clapping
creating ./ESC-50-wav/303 - Clapping
creating ./ESC-50-wav/303 - Clapping
creating ./ESC-50-wav/303 - Clapping
creating ./ESC-50-wav/301 - Crying baby
You got a short sound file 301 - Crying baby/2-107351-A.ogg with shape (109440,)!
zero padding data to shape (110250,)
creating ./ESC-50-wav/301 - Crying baby
creating ./ESC-50-wav/301 - Crying baby
creating ./ESC-50-wav/301 - Crying baby
creating ./ESC-50-wav/301 - Crying baby
creating ./ESC-50-wav/301 - Crying baby
creating ./ESC-50-wav/301 - Crying baby
creating ./ESC-50-wav/301 - Crying baby
creating ./ESC-50-wav/301 - Crying baby
creating ./ESC-50-wav/301 - Crying baby
creating ./ESC-50-wav/301 - Crying baby
creating ./ESC-50-wav/301 - Crying baby
creating ./ESC-50-wav/301 - Crying baby
creating ./ESC-50-wav/301 - 

creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click
creating ./ESC-50-wav/402 - Mouse click


### Ok now we will transform the wav files into spectrograms

In [5]:
# First choose the desired transform
transform = "stft" #choose the signal processing technique: ["stft","mel","cqt","cwt","mfcc"]

if transform == "stft":
    # location to write the linear spectrogram files (converted from wave files)
    K_SPECTDIR = main_dir + 'stft'
elif transform == "mel":
    # location to write the mel spec files (converted from wave files)
    K_SPECTDIR = main_dir + 'mel'
elif transform == "cqt":
    # location to write the CQT spec files (converted from wave files)
    K_SPECTDIR = main_dir + 'cqt'
elif transform == "cwt":
    # location to write the wavelet files (converted from wave files)
    K_SPECTDIR = main_dir + 'cwt'
elif transform == "mfcc":
    # location to write the mfcc files (converted from wave files)
    K_SPECTDIR = main_dir + 'mfcc'
else:
    raise ValueError("Transform not supported! Please choose from ['stft','mel','cqt','cwt','mfcc']")

In [6]:
#routines to convert wav to spectrograms using Librosa as backend. For wavelets we are using Pywavelet library.
import scipy
from PIL import TiffImagePlugin
from PIL import Image


def wav2stft(fname, srate, fftSize, fftHop, y_scale='linear', dur=None, showplt=False, dcbin=True) :
    try:
        audiodata, samplerate = librosa.load(fname, sr=srate, mono=True, duration=dur) 
    except:
        print('can not read ' + fname)
        return
    
    S = np.abs(librosa.stft(audiodata, n_fft=fftSize, hop_length=fftHop, win_length=fftSize,  center=False))

    if (dcbin ==  False) :
        S = np.delete(S, (0), axis=0)  # delete freq 0 row
            #note: a pure DC input signal bleeds into bin 1, too.

    D = librosa.amplitude_to_db(S, ref=np.max)
    
    if showplt : # Dangerous for long runs - it opens a new figure for each file!
        librosa.display.specshow(D, y_axis=y_scale, x_axis='time', sr=srate, hop_length=fftHop)
        plt.colorbar(format='%+2.0f dB')
        plt.title(showplt)
        plt.show(block=True)
                
    return D

def wav2cqt(fname, srate, fftSize, fftHop, dur=None, showplt=False) :
    try:
        audiodata, samplerate = librosa.load(fname, sr=srate, mono=True, duration=dur) 
    except:
        print('can not read ' + fname)
        return
    
    S = librosa.cqt(audiodata,hop_length=fftHop,n_bins=fftSize//2,bins_per_octave=128)

    D = librosa.amplitude_to_db(S, ref=np.max)
    
    if showplt : # Dangerous for long runs - it opens a new figure for each file!
        librosa.display.specshow(D, y_axis='cqt_note', x_axis='time', sr=srate, hop_length=fftHop)
        plt.colorbar(format='%+2.0f dB')
        plt.title(showplt)
        plt.show(block=True)
                
    return D

def wav2cwt(fname, srate, freq_bins, wavelet='morl', dur=None, showplt=False):
    import pywt
    try:
        audiodata, samplerate = librosa.load(fname, sr=srate, mono=True, duration=dur) 
    except:
        print('can not read ' + fname)
        return
    widths = np.arange(1, freq_bins+1) #no.of freq bins
    
    S, freqs = pywt.cwt(audiodata, widths, wavelet, sampling_period=1/srate)

    D = librosa.amplitude_to_db(S, ref=np.max)
    
    if showplt : # Dangerous for long runs - it opens a new figure for each file!
        librosa.display.specshow(D, y_axis='linear', x_axis='time', sr=srate, hop_length=fftHop)
        plt.colorbar(format='%+2.0f dB')
        plt.title(showplt)
        plt.show(block=True)
                
    return D

def wav2mel(fname, srate, fftSize, fftHop, n_mels=128, dur=None, showplt=False):
    try:
        audiodata, samplerate = librosa.load(fname, sr=srate, mono=True, duration=dur) 
    except:
        print('can not read ' + fname)
        return
    
    S = librosa.feature.melspectrogram(audiodata, sr=srate, n_fft=fftSize, hop_length=fftHop, n_mels=n_mels)

    D = librosa.power_to_db(S, ref=np.max)
    
    if showplt : # Dangerous for long runs - it opens a new figure for each file!
        librosa.display.specshow(D, y_axis='mel', x_axis='time', sr=srate, hop_length=fftHop)
        plt.colorbar(format='%+2.0f dB')
        plt.title(showplt)
        plt.show(block=True)
                
    return D

def wav2mfcc(fname, srate, fftSize, fftHop, n_mfcc=128, dur=None, showplt=False):
    try:
        audiodata, samplerate = librosa.load(fname, sr=srate, mono=True, duration=dur) 
    except:
        print('can not read ' + fname)
        return
    
    S = librosa.feature.mfcc(audiodata,sr=srate, n_fft=fftSize, hop_length=fftHop, n_mfcc=n_mfcc)
    
    if showplt : # Dangerous for long runs - it opens a new figure for each file!
        librosa.display.specshow(D, y_axis='linear', x_axis='time', sr=srate, hop_length=fftHop)
        plt.colorbar(format='%+2.0f dB')
        plt.title(showplt)
        plt.show(block=True)
                
    return S  

In [7]:
def esc50get_fold(string):
    """get fold no. from ESC-50 dataset using the filename. Labels #1-5"""
    label_pos = string.index("-")
    return string[label_pos-1]

In [8]:
def wav2Spect(topdir, outdir, dur, srate, fftSize, fftHop, transform, neww=None, newh=None, newscale=None) :
    """ 
        Creates spectrograms for subfolder-labeled wavfiles. 
        Creates class folders for the spectrogram files in outdir with the same structure found in topdir.
        Outputs both tif files (greater resolution) and png files (for TFRecords)
        
        Parameters
            topdir - the dir containing class folders containing wav files. 
            outdir - the top level directory to write wave files to (written in to class subfolders)
            dur - (in seconds) all files will be truncated or zeropadded to have this duration given the srate
            srate - input files will be resampled to srate as they are read in before being saved as wav files
            transform - desired transform, defined at the top of this notebook
            
        Scaling Params
            neww - scale width of spectrogram (in pixels) to this value 
            newh - scale height of spectrogram (in pixels) to this value. maintains aspect ratio if only neww is specified
            *if both neww and newh is left unspecified no scaling is carried out
            newscale - normalize intensity values of spectrogram from about [-80,0]dB to a new range. default is [0,255])
    """ 
    if newscale is None:
        newscale = [0,255]
    if newscale[0] != 0 or newscale[1] != 255:
        warnings.warn("Scaling of intensity values outside unit8 range.Change newscale param to  [0,255] else PNG files will not be generated!")
    
    subdirs = get_subdirs(topdir)
    count = 0
    for subdir in subdirs:
    
        fullpaths, _ = listDirectory(topdir + '/' + subdir, '.wav') 
        
        for idx in range(len(fullpaths)) : 
            fname = os.path.basename(fullpaths[idx])
            fold = esc50get_fold(fname)
            
            if transform == "stft":
                D = wav2stft(fullpaths[idx], srate, fftSize, fftHop)
            elif transform == "mel":
                D = wav2mel(fullpaths[idx], srate, fftSize, fftHop)
            elif transform == "cqt":
                D = wav2cqt(fullpaths[idx], srate, fftSize, fftHop)
            elif transform == "cwt":
                D = wav2cwt(fullpaths[idx], srate, 64, 'morl')
            elif transform == "mfcc":
                D = wav2mfcc(fullpaths[idx], srate, fftSize, fftHop)
            
            try:
                os.stat(outdir + '_tif/' + fold + '/' + subdir) # test for existence
            except:
                os.makedirs(outdir + '_tif/' + fold + '/' + subdir) # create if necessary
            
            tiffspect.specScale(D, outdir+'_tif/'+fold+'/'+subdir+'/'+os.path.splitext(fname)[0],
                                neww, newh, newscale)
            
            if newscale[0] == 0 and newscale[1] == 255:
            # output png if scale is correct
                try:
                    os.stat(outdir + '_png/' + fold + '/' + subdir) # test for existence
                except:
                    os.makedirs(outdir + '_png/' + fold + '/' + subdir) # create if necessary
                
                tiffspect.tif2png(outdir+'_tif/'+fold+'/'+subdir+'/'+os.path.splitext(fname)[0],
                                    outdir+'_png/'+fold+'/'+subdir+'/'+os.path.splitext(fname)[0])
            
            print(str(count) + ': ' + subdir + '/' + os.path.splitext(fname)[0])
            count +=1
    print("COMPLETE")     
 

In [9]:
wav2Spect(K_WAVEDIR, K_SPECTDIR, K_DUR, K_SR, K_FFTSIZE, K_HOP, transform)

0: 103 - Pig/4-250869-C
1: 103 - Pig/1-208757-C
2: 103 - Pig/3-253084-C
3: 103 - Pig/4-250869-A
4: 103 - Pig/2-158746-D
5: 103 - Pig/4-132383-A
6: 103 - Pig/5-127990-A
7: 103 - Pig/2-37870-A
8: 103 - Pig/5-103416-A
9: 103 - Pig/5-103420-A
10: 103 - Pig/2-166644-C
11: 103 - Pig/2-158746-A
12: 103 - Pig/4-147240-A
13: 103 - Pig/4-250869-B
14: 103 - Pig/3-253084-B
15: 103 - Pig/3-253081-A
16: 103 - Pig/1-208757-D
17: 103 - Pig/3-253084-A
18: 103 - Pig/4-132383-B
19: 103 - Pig/3-233151-A
20: 103 - Pig/5-103415-A
21: 103 - Pig/1-260640-B
22: 103 - Pig/1-208757-E
23: 103 - Pig/3-253084-E
24: 103 - Pig/1-208757-A
25: 103 - Pig/1-260640-C
26: 103 - Pig/5-103422-A
27: 103 - Pig/5-117250-A
28: 103 - Pig/2-166644-A
29: 103 - Pig/4-147240-B
30: 103 - Pig/2-166644-B
31: 103 - Pig/2-158746-C
32: 103 - Pig/4-234644-A
33: 103 - Pig/2-158746-B
34: 103 - Pig/1-260640-A
35: 103 - Pig/5-103421-A
36: 103 - Pig/3-253084-D
37: 103 - Pig/5-103418-A
38: 103 - Pig/3-257858-A
39: 103 - Pig/1-208757-B
40: 410 - G

262: 109 - Sheep/2-196688-A
263: 109 - Sheep/5-198891-C
264: 109 - Sheep/5-198891-B
265: 109 - Sheep/3-197408-A
266: 109 - Sheep/2-119161-A
267: 109 - Sheep/4-196671-B
268: 109 - Sheep/2-182508-A
269: 109 - Sheep/5-200329-A
270: 109 - Sheep/4-196672-A
271: 109 - Sheep/1-57795-A
272: 109 - Sheep/3-20861-A
273: 109 - Sheep/5-200329-B
274: 109 - Sheep/4-250864-A
275: 109 - Sheep/2-119161-B
276: 109 - Sheep/1-49409-B
277: 109 - Sheep/4-188703-A
278: 109 - Sheep/3-197408-B
279: 109 - Sheep/5-61635-A
280: 202 - Sea waves/3-144827-A
281: 202 - Sea waves/3-144827-B
282: 202 - Sea waves/3-166422-A
283: 202 - Sea waves/1-43760-A
284: 202 - Sea waves/2-102852-A
285: 202 - Sea waves/3-187710-A
286: 202 - Sea waves/4-182613-B
287: 202 - Sea waves/4-204618-A
288: 202 - Sea waves/2-155801-A
289: 202 - Sea waves/1-39901-A
290: 202 - Sea waves/5-208810-A
291: 202 - Sea waves/2-132157-B
292: 202 - Sea waves/1-28135-B
293: 202 - Sea waves/2-133863-A
294: 202 - Sea waves/5-213077-A
295: 202 - Sea waves/2-

515: 407 - Vacuum cleaner/5-182007-A
516: 407 - Vacuum cleaner/2-141681-B
517: 407 - Vacuum cleaner/5-212059-A
518: 407 - Vacuum cleaner/1-100210-B
519: 407 - Vacuum cleaner/2-141682-B
520: 105 - Frog/5-156026-A
521: 105 - Frog/2-52085-B
522: 105 - Frog/4-154793-A
523: 105 - Frog/5-156026-B
524: 105 - Frog/4-99644-B
525: 105 - Frog/2-32515-C
526: 105 - Frog/4-99193-B
527: 105 - Frog/3-71964-C
528: 105 - Frog/1-18755-A
529: 105 - Frog/3-71964-B
530: 105 - Frog/4-99644-C
531: 105 - Frog/3-70962-B
532: 105 - Frog/3-70962-C
533: 105 - Frog/1-18755-B
534: 105 - Frog/2-52085-A
535: 105 - Frog/5-156026-D
536: 105 - Frog/2-32515-D
537: 105 - Frog/1-15689-A
538: 105 - Frog/1-31836-B
539: 105 - Frog/2-32515-A
540: 105 - Frog/1-18757-A
541: 105 - Frog/1-15689-B
542: 105 - Frog/1-31836-A
543: 105 - Frog/3-102908-A
544: 105 - Frog/3-71964-A
545: 105 - Frog/5-187201-B
546: 105 - Frog/1-17970-A
547: 105 - Frog/5-156026-C
548: 105 - Frog/5-187201-A
549: 105 - Frog/3-83527-A
550: 105 - Frog/2-32834-A
5

780: 108 - Insects/4-182314-B
781: 108 - Insects/1-75189-A
782: 108 - Insects/5-198278-C
783: 108 - Insects/2-160128-A
784: 108 - Insects/5-198278-A
785: 108 - Insects/4-143118-A
786: 108 - Insects/3-104761-A
787: 108 - Insects/5-197446-A
788: 108 - Insects/2-99955-C
789: 108 - Insects/4-182314-A
790: 108 - Insects/1-17585-A
791: 108 - Insects/3-110913-C
792: 108 - Insects/3-105236-A
793: 108 - Insects/1-19501-A
794: 108 - Insects/3-105235-A
795: 108 - Insects/4-143118-B
796: 108 - Insects/5-233787-A
797: 108 - Insects/2-81970-C
798: 108 - Insects/2-81970-A
799: 108 - Insects/1-46938-B
800: 508 - Airplane/1-11687-A
801: 508 - Airplane/2-78799-A
802: 508 - Airplane/4-161099-A
803: 508 - Airplane/2-78781-A
804: 508 - Airplane/4-161105-B
805: 508 - Airplane/5-215445-A
806: 508 - Airplane/4-161105-A
807: 508 - Airplane/1-53467-A
808: 508 - Airplane/2-160888-A
809: 508 - Airplane/2-74361-A
810: 508 - Airplane/2-105270-A
811: 508 - Airplane/5-215447-A
812: 508 - Airplane/3-143560-A
813: 508 

1048: 204 - Crickets/5-210540-A
1049: 204 - Crickets/4-195805-A
1050: 204 - Crickets/3-162786-A
1051: 204 - Crickets/3-129678-A
1052: 204 - Crickets/2-96033-A
1053: 204 - Crickets/3-146033-A
1054: 204 - Crickets/3-163607-A
1055: 204 - Crickets/5-215179-A
1056: 204 - Crickets/4-195707-A
1057: 204 - Crickets/2-125821-A
1058: 204 - Crickets/1-9841-A
1059: 204 - Crickets/4-172143-A
1060: 204 - Crickets/4-171823-A
1061: 204 - Crickets/5-216214-A
1062: 204 - Crickets/4-202749-A
1063: 204 - Crickets/1-59102-A
1064: 204 - Crickets/4-194246-A
1065: 204 - Crickets/5-215172-A
1066: 204 - Crickets/1-57318-A
1067: 204 - Crickets/1-7456-A
1068: 204 - Crickets/3-155766-A
1069: 204 - Crickets/4-163697-A
1070: 204 - Crickets/2-110613-A
1071: 204 - Crickets/2-85139-A
1072: 204 - Crickets/1-65483-A
1073: 204 - Crickets/2-125875-A
1074: 204 - Crickets/3-129338-A
1075: 204 - Crickets/5-204352-A
1076: 204 - Crickets/2-83270-A
1077: 204 - Crickets/5-216213-A
1078: 204 - Crickets/4-181362-A
1079: 204 - Cricke

1295: 404 - Door - wood creaks/5-161270-A
1296: 404 - Door - wood creaks/4-108352-A
1297: 404 - Door - wood creaks/3-94355-A
1298: 404 - Door - wood creaks/1-51805-C
1299: 404 - Door - wood creaks/2-91912-B
1300: 404 - Door - wood creaks/5-173568-A
1301: 404 - Door - wood creaks/1-51805-E
1302: 404 - Door - wood creaks/3-101381-B
1303: 404 - Door - wood creaks/3-103401-C
1304: 404 - Door - wood creaks/1-51805-A
1305: 404 - Door - wood creaks/2-7321-A
1306: 404 - Door - wood creaks/2-70367-A
1307: 404 - Door - wood creaks/4-107122-A
1308: 404 - Door - wood creaks/3-103401-D
1309: 404 - Door - wood creaks/1-51805-F
1310: 404 - Door - wood creaks/2-91912-A
1311: 404 - Door - wood creaks/4-160036-B
1312: 404 - Door - wood creaks/3-103401-B
1313: 404 - Door - wood creaks/1-51805-H
1314: 404 - Door - wood creaks/2-87780-A
1315: 404 - Door - wood creaks/3-118194-A
1316: 404 - Door - wood creaks/5-177779-A
1317: 404 - Door - wood creaks/4-128659-A
1318: 404 - Door - wood creaks/5-161270-B
1319

1543: 205 - Chirping birds/1-54918-A
1544: 205 - Chirping birds/1-100038-A
1545: 205 - Chirping birds/5-242491-A
1546: 205 - Chirping birds/2-108760-B
1547: 205 - Chirping birds/2-108761-A
1548: 205 - Chirping birds/5-243459-A
1549: 205 - Chirping birds/1-46040-A
1550: 205 - Chirping birds/5-242490-A
1551: 205 - Chirping birds/2-72547-C
1552: 205 - Chirping birds/4-216211-A
1553: 205 - Chirping birds/2-122616-A
1554: 205 - Chirping birds/2-72547-D
1555: 205 - Chirping birds/3-156581-B
1556: 205 - Chirping birds/1-38560-A
1557: 205 - Chirping birds/5-243448-A
1558: 205 - Chirping birds/3-156581-A
1559: 205 - Chirping birds/2-108760-A
1560: 510 - Hand saw/5-253094-C
1561: 510 - Hand saw/4-102844-C
1562: 510 - Hand saw/4-102844-B
1563: 510 - Hand saw/2-59321-A
1564: 510 - Hand saw/5-253101-A
1565: 510 - Hand saw/3-102583-B
1566: 510 - Hand saw/1-7974-B
1567: 510 - Hand saw/3-96606-B
1568: 510 - Hand saw/4-102844-A
1569: 510 - Hand saw/4-198360-C
1570: 510 - Hand saw/2-51630-A
1571: 510 - 

1797: 209 - Toilet flush/2-70280-A
1798: 209 - Toilet flush/2-74977-A
1799: 209 - Toilet flush/3-92637-A
1800: 506 - Train/3-141559-A
1801: 506 - Train/4-165791-A
1802: 506 - Train/5-199284-B
1803: 506 - Train/1-64473-A
1804: 506 - Train/1-88409-A
1805: 506 - Train/4-165845-B
1806: 506 - Train/1-54065-B
1807: 506 - Train/2-122067-A
1808: 506 - Train/5-197121-A
1809: 506 - Train/1-119125-A
1810: 506 - Train/2-122066-A
1811: 506 - Train/4-178881-A
1812: 506 - Train/3-159445-B
1813: 506 - Train/1-70300-A
1814: 506 - Train/5-188945-A
1815: 506 - Train/2-135649-C
1816: 506 - Train/4-168446-A
1817: 506 - Train/5-198600-A
1818: 506 - Train/3-138212-A
1819: 506 - Train/5-197121-B
1820: 506 - Train/2-135649-A
1821: 506 - Train/3-159445-A
1822: 506 - Train/4-165791-B
1823: 506 - Train/1-62509-A
1824: 506 - Train/2-122066-B
1825: 506 - Train/2-135649-B
1826: 506 - Train/3-157492-A
1827: 506 - Train/2-122067-B
1828: 506 - Train/5-188796-A
1829: 506 - Train/4-165845-A
1830: 506 - Train/3-135650-A
1

In [10]:
# output human readable class labels into text file (required for TFRecords)
dirs2labelfile(K_SPECTDIR+'_png/1', main_dir+'/labels.txt')

### We are done...alternative CQT transform below, which reads in a linear STFT spectrogram, undoes the intensity normalization, then does the conversion

In [11]:
# alternative CQT transform below, which converts from a linear spectrogram
# from lonce wyse: https://github.com/lonce/audioST/blob/master/dataPrep/ESC50_Convert.ipynb

K_QSPECTDIR = main_dir + 'esc50Qspect' #directory to output files

def logfmap(I, L, H) :
    """
    % [M,N] = logfmap(I,L,H)
    I - number of rows in the original spectrogram
    L - low bin to preserve
    H - high bin to preserver
    
    %     Return a maxtrix for premultiplying spectrograms to map
    %     the rows into a log frequency space.
    %     Output map covers bins L to H of input
    %     L must be larger than 1, since the lowest bin of the FFT
    %     (corresponding to 0 Hz) cannot be represented on a 
    %     log frequency axis.  Including bins close to 1 makes 
    %     the number of output rows exponentially larger.
    %     N returns the recovery matrix such that N*M is approximately I
    %     (for dimensions L to H).
    %     
    % Ported from MATLAB code written by Dan Ellis:
    % 2004-05-21 dpwe@ee.columbia.edu
    """
    ratio = (H-1)/H;
    opr = np.int(np.round(np.log(L/H)/np.log(ratio))) #number of frequency bins in log rep + 1
    print('opr is ' + str(opr))
    ibin = L*np.exp(list(range(0,opr)*-np.log(ratio)))  #fractional bin numbers (len(ibin) = opr-1)
    
    M = np.zeros((opr,I))
    eps=np.finfo(float).eps
    
    for i in range(0, opr) :
        # Where do we sample this output bin?
        # Idea is to make them 1:1 at top, and progressively denser below
        # i.e. i = max -> bin = topbin, i = max-1 -> bin = topbin-1, 
        # but general form is bin = A exp (i/B)

        tt = np.multiply(np.pi, (list(range(0,I))-ibin[i]))
        M[i,:] = np.divide((np.sin(tt)+eps) , (tt+eps));

    # Normalize rows, but only if they are boosted by the operation
    G = np.ones((I));
    print ('H is ' + str(H))
    G[0:H] = np.divide(list(range(0,H)), H)
    
    N = np.transpose(np.multiply(M,np.matlib.repmat(G,opr,1)))
                   
    return M,N

In [12]:
def esc50Spect2logFreqSpect(topdir, outdir, srate, fftSize, fftHop, lowRow=1, neww=None, newh=None, newscale=None) :
    """ 
        Creates psuedo constant-Q spectrograms from linear frequency spectrograms. 
        Creates class foldersfna in outdir with the same structure found in topdir.
        
        Parameters
            topdir - the dir containing class folders containing tif (log magnitude) spectrogram files. 
            outdir - the top level directory to write psuedo constantQ files to (written in to class subfolders)
            lowRow is the lowest row in the FFT that you want to include in the psuedo constant Q spectrogram
    """ 
    
    # First lets get the logf map we want
    LIN_FREQ_BINS = int(fftSize/2+1) #number of bins in original linear frequency mag spectrogram
    LOW_ROW = lowRow
    LOG_FREQ_BINS = int(fftSize/2+1) #resample the lgfmapped psuedo consantQ matrix to have this many frequency bins
    M,N = logfmap(LIN_FREQ_BINS,LOW_ROW,LOG_FREQ_BINS);
    
    if newscale is None:
        newscale = [0,255]
    if newscale[0] != 0 or newscale[1] != 255:
        warnings.warn("Scaling of intensity values outside unit8 range.Change newscale param to  [0,255] else PNG files will not be generated!")
    
    folds = get_subdirs(topdir)
    count = 0
    for fold in folds:
        subdir = get_subdirs(topdir + '/' + fold)
        for classes in subdir:
            try:
                os.stat(outdir + '_tif/' + fold + '/' + classes) # test for existence
            except:
                os.makedirs(outdir + '_tif/' + fold + '/' + classes) # create if necessary
            fullpaths, _ = listDirectory(topdir + '/' + fold + '/' + classes, '.tif')

            for idx in range(len(fullpaths)) : 
                fname = os.path.basename(fullpaths[idx])
                D, tiffinfo = tiffspect.invSpecScale(fullpaths[idx])            
                #print('tiffinfo is ' + str(tiffinfo))

                # Here's the beef
                MD = np.dot(M,D);
                MD = scipy.signal.resample(MD, LIN_FREQ_BINS) #downsample to something reasonable 

                #save
                tiffinfo["linFreqBins"] =  LIN_FREQ_BINS
                tiffinfo["lowRow"] = LOW_ROW
                tiffinfo["logFreqBins"] = LOG_FREQ_BINS
                tiffspect.specScale(MD, outdir+'_tif/'+fold+'/'+classes+'/'+os.path.splitext(fname)[0],
                                    neww, newh, newscale, lwinfo=tiffinfo)
                
                if newscale[0] == 0 and newscale[1] == 255:
                # output png if scale is correct
                    try:
                        os.stat(outdir + '_png/' + fold + '/' + classes) # test for existence
                    except:
                        os.makedirs(outdir + '_png/' + fold + '/' + classes) # create if necessary

                    tiffspect.tif2png(outdir+'_tif/'+fold+'/'+classes+'/'+os.path.splitext(fname)[0],
                                        outdir+'_png/'+fold+'/'+classes+'/'+os.path.splitext(fname)[0])                
                
                print(str(count) + ': ' + classes + '/' + os.path.splitext(fname)[0])
                count +=1
    print("COMPLETE") 

In [13]:
esc50Spect2logFreqSpect(K_SPECTDIR+'_tif', K_QSPECTDIR, K_SR, K_FFTSIZE, K_HOP, lowRow=10)

opr is 2018
H is 513
0: 103 - Pig/2-166644-A
1: 103 - Pig/2-158746-A
2: 103 - Pig/2-166644-C
3: 103 - Pig/2-166644-B
4: 103 - Pig/2-158746-D
5: 103 - Pig/2-158746-B
6: 103 - Pig/2-158746-C
7: 103 - Pig/2-37870-A
8: 410 - Glass breaking/2-141563-A
9: 410 - Glass breaking/2-173559-A
10: 410 - Glass breaking/2-173618-A
11: 410 - Glass breaking/2-106881-A
12: 410 - Glass breaking/2-250710-A
13: 410 - Glass breaking/2-112213-A
14: 410 - Glass breaking/2-173607-A
15: 410 - Glass breaking/2-112213-B
16: 505 - Engine/2-106487-A
17: 505 - Engine/2-107228-B
18: 505 - Engine/2-78651-A
19: 505 - Engine/2-106015-B
20: 505 - Engine/2-107228-A
21: 505 - Engine/2-106014-A
22: 505 - Engine/2-106015-A
23: 505 - Engine/2-106486-A
24: 401 - Door knock/2-120218-B
25: 401 - Door knock/2-118624-A
26: 401 - Door knock/2-134915-A
27: 401 - Door knock/2-114254-A
28: 401 - Door knock/2-133889-A
29: 401 - Door knock/2-140841-A
30: 401 - Door knock/2-120218-A
31: 401 - Door knock/2-118625-A
32: 302 - Sneezing/2-13

262: 404 - Door - wood creaks/2-87780-A
263: 404 - Door - wood creaks/2-123712-A
264: 207 - Wind/2-109371-D
265: 207 - Wind/2-109371-A
266: 207 - Wind/2-104952-B
267: 207 - Wind/2-104952-A
268: 207 - Wind/2-109374-A
269: 207 - Wind/2-205966-A
270: 207 - Wind/2-109371-C
271: 207 - Wind/2-109371-B
272: 502 - Chainsaw/2-50668-B
273: 502 - Chainsaw/2-68391-A
274: 502 - Chainsaw/2-50667-B
275: 502 - Chainsaw/2-68391-B
276: 502 - Chainsaw/2-77945-B
277: 502 - Chainsaw/2-77945-A
278: 502 - Chainsaw/2-50667-A
279: 502 - Chainsaw/2-50668-A
280: 309 - Snoring/2-110417-B
281: 309 - Snoring/2-52001-A
282: 309 - Snoring/2-80313-A
283: 309 - Snoring/2-52001-B
284: 309 - Snoring/2-114609-A
285: 309 - Snoring/2-117330-A
286: 309 - Snoring/2-114609-B
287: 309 - Snoring/2-110417-A
288: 501 - Helicopter/2-188822-C
289: 501 - Helicopter/2-37806-B
290: 501 - Helicopter/2-37806-A
291: 501 - Helicopter/2-188822-A
292: 501 - Helicopter/2-188822-B
293: 501 - Helicopter/2-37806-C
294: 501 - Helicopter/2-37806-D

522: 210 - Thunderstorm/4-161519-A
523: 210 - Thunderstorm/4-165813-A
524: 210 - Thunderstorm/4-156993-A
525: 210 - Thunderstorm/4-125070-A
526: 210 - Thunderstorm/4-165813-B
527: 210 - Thunderstorm/4-125071-A
528: 203 - Crackling fire/4-182368-A
529: 203 - Crackling fire/4-182369-A
530: 203 - Crackling fire/4-164661-A
531: 203 - Crackling fire/4-170247-B
532: 203 - Crackling fire/4-170247-A
533: 203 - Crackling fire/4-181563-A
534: 203 - Crackling fire/4-171207-A
535: 203 - Crackling fire/4-164661-B
536: 409 - Clock tick/4-181865-A
537: 409 - Clock tick/4-181035-A
538: 409 - Clock tick/4-188033-A
539: 409 - Clock tick/4-198965-A
540: 409 - Clock tick/4-191327-A
541: 409 - Clock tick/4-194711-A
542: 409 - Clock tick/4-175945-A
543: 409 - Clock tick/4-179984-A
544: 201 - Rain/4-161127-A
545: 201 - Rain/4-180380-A
546: 201 - Rain/4-166661-A
547: 201 - Rain/4-177250-A
548: 201 - Rain/4-181286-A
549: 201 - Rain/4-163264-A
550: 201 - Rain/4-164206-A
551: 201 - Rain/4-160999-A
552: 108 - Ins

780: 106 - Cat/4-149940-A
781: 106 - Cat/4-149940-B
782: 106 - Cat/4-133047-A
783: 106 - Cat/4-133047-C
784: 402 - Mouse click/4-192068-A
785: 402 - Mouse click/4-195305-A
786: 402 - Mouse click/4-204683-A
787: 402 - Mouse click/4-202298-A
788: 402 - Mouse click/4-204612-A
789: 402 - Mouse click/4-210309-A
790: 402 - Mouse click/4-201800-A
791: 402 - Mouse click/4-201300-A
792: 310 - Drinking - sipping/4-188595-A
793: 310 - Drinking - sipping/4-213193-A
794: 310 - Drinking - sipping/4-210593-A
795: 310 - Drinking - sipping/4-204202-A
796: 310 - Drinking - sipping/4-194808-A
797: 310 - Drinking - sipping/4-188191-B
798: 310 - Drinking - sipping/4-188191-A
799: 310 - Drinking - sipping/4-188191-C
800: 103 - Pig/5-103415-A
801: 103 - Pig/5-103422-A
802: 103 - Pig/5-127990-A
803: 103 - Pig/5-103418-A
804: 103 - Pig/5-103420-A
805: 103 - Pig/5-103421-A
806: 103 - Pig/5-117250-A
807: 103 - Pig/5-103416-A
808: 410 - Glass breaking/5-233607-A
809: 410 - Glass breaking/5-233605-A
810: 410 - Gla

1035: 503 - Siren/5-117118-A
1036: 503 - Siren/5-117122-A
1037: 503 - Siren/5-184323-A
1038: 503 - Siren/5-133989-B
1039: 503 - Siren/5-160551-A
1040: 206 - Water drops/5-261433-A
1041: 206 - Water drops/5-259180-A
1042: 206 - Water drops/5-254832-A
1043: 206 - Water drops/5-262641-A
1044: 206 - Water drops/5-261439-A
1045: 206 - Water drops/5-241846-A
1046: 206 - Water drops/5-257349-A
1047: 206 - Water drops/5-254832-B
1048: 408 - Clock alarm/5-219242-B
1049: 408 - Clock alarm/5-219242-A
1050: 408 - Clock alarm/5-234247-A
1051: 408 - Clock alarm/5-211408-A
1052: 408 - Clock alarm/5-250629-A
1053: 408 - Clock alarm/5-233645-A
1054: 408 - Clock alarm/5-223176-A
1055: 408 - Clock alarm/5-210612-A
1056: 404 - Door - wood creaks/5-177779-A
1057: 404 - Door - wood creaks/5-191497-A
1058: 404 - Door - wood creaks/5-173568-A
1059: 404 - Door - wood creaks/5-188606-A
1060: 404 - Door - wood creaks/5-181458-A
1061: 404 - Door - wood creaks/5-187444-A
1062: 404 - Door - wood creaks/5-161270-A
1

1281: 507 - Church bells/3-98869-A
1282: 507 - Church bells/3-111102-B
1283: 507 - Church bells/3-98193-A
1284: 507 - Church bells/3-87936-A
1285: 507 - Church bells/3-139109-A
1286: 507 - Church bells/3-122110-A
1287: 507 - Church bells/3-111102-A
1288: 104 - Cow/3-160993-A
1289: 104 - Cow/3-126358-A
1290: 104 - Cow/3-124376-B
1291: 104 - Cow/3-152039-B
1292: 104 - Cow/3-152039-A
1293: 104 - Cow/3-163727-A
1294: 104 - Cow/3-126358-B
1295: 104 - Cow/3-124376-A
1296: 407 - Vacuum cleaner/3-159346-A
1297: 407 - Vacuum cleaner/3-159347-B
1298: 407 - Vacuum cleaner/3-152020-B
1299: 407 - Vacuum cleaner/3-159348-A
1300: 407 - Vacuum cleaner/3-152020-A
1301: 407 - Vacuum cleaner/3-152020-C
1302: 407 - Vacuum cleaner/3-159347-A
1303: 407 - Vacuum cleaner/3-159346-B
1304: 105 - Frog/3-71964-A
1305: 105 - Frog/3-70962-B
1306: 105 - Frog/3-102908-A
1307: 105 - Frog/3-71964-C
1308: 105 - Frog/3-70962-A
1309: 105 - Frog/3-83527-A
1310: 105 - Frog/3-70962-C
1311: 105 - Frog/3-71964-B
1312: 102 - Ro

1532: 307 - Laughing/3-118487-A
1533: 307 - Laughing/3-110536-A
1534: 307 - Laughing/3-119459-A
1535: 307 - Laughing/3-152912-A
1536: 303 - Clapping/3-197435-A
1537: 303 - Clapping/3-181278-A
1538: 303 - Clapping/3-177083-A
1539: 303 - Clapping/3-177082-A
1540: 303 - Clapping/3-149465-A
1541: 303 - Clapping/3-130330-A
1542: 303 - Clapping/3-138114-A
1543: 303 - Clapping/3-197435-B
1544: 301 - Crying baby/3-152007-C
1545: 301 - Crying baby/3-152007-E
1546: 301 - Crying baby/3-152007-B
1547: 301 - Crying baby/3-152007-D
1548: 301 - Crying baby/3-151081-A
1549: 301 - Crying baby/3-151080-A
1550: 301 - Crying baby/3-152007-A
1551: 301 - Crying baby/3-151081-B
1552: 209 - Toilet flush/3-100018-A
1553: 209 - Toilet flush/3-108677-A
1554: 209 - Toilet flush/3-98771-A
1555: 209 - Toilet flush/3-112356-A
1556: 209 - Toilet flush/3-118059-A
1557: 209 - Toilet flush/3-92637-A
1558: 209 - Toilet flush/3-108791-A
1559: 209 - Toilet flush/3-93010-A
1560: 506 - Train/3-135650-B
1561: 506 - Train/3-13

1786: 308 - Brushing teeth/1-45641-A
1787: 308 - Brushing teeth/1-58923-A
1788: 308 - Brushing teeth/1-58923-B
1789: 308 - Brushing teeth/1-17092-A
1790: 308 - Brushing teeth/1-61534-A
1791: 308 - Brushing teeth/1-67432-A
1792: 306 - Footsteps/1-155858-B
1793: 306 - Footsteps/1-155858-C
1794: 306 - Footsteps/1-155858-D
1795: 306 - Footsteps/1-155858-F
1796: 306 - Footsteps/1-155858-A
1797: 306 - Footsteps/1-155858-E
1798: 306 - Footsteps/1-223162-A
1799: 306 - Footsteps/1-51147-A
1800: 110 - Crow/1-39835-A
1801: 110 - Crow/1-56234-A
1802: 110 - Crow/1-103298-A
1803: 110 - Crow/1-75162-A
1804: 110 - Crow/1-96950-B
1805: 110 - Crow/1-96950-A
1806: 110 - Crow/1-39835-B
1807: 110 - Crow/1-56233-A
1808: 204 - Crickets/1-9841-A
1809: 204 - Crickets/1-7456-A
1810: 204 - Crickets/1-57316-A
1811: 204 - Crickets/1-65483-A
1812: 204 - Crickets/1-80840-A
1813: 204 - Crickets/1-80841-A
1814: 204 - Crickets/1-59102-A
1815: 204 - Crickets/1-57318-A
1816: 208 - Pouring water/1-50688-A
1817: 208 - Pour

In [14]:
## example of using librosa spectrogram viewer as well as tiff file writer and reader
# import scipy
# %matplotlib inline

# # Just run this on any wav file in your system.
# audiodata, samplerate = librosa.load('esc50Wave/109 - Sheep/4-196671-B.wav', sr=K_SR, mono=True, duration=K_DUR) # resamples if necessary (some esc-50 files are in 48K)
# #audiodata, samplerate = librosa.load(K_WAVEDIR + '/1/BeingRural_short.wav', sr=K_SR, mono=True, duration=K_DUR) # resamples if necessary (some esc-50 files are in 48K)

# print('audiodata max is ' + str(np.max(audiodata)) + ', and audiodata sum is ' + str(np.sum(audiodata)))

# # compute spectrogram and display
# S = np.abs(librosa.stft(audiodata, n_fft=K_FFTSIZE, hop_length=K_HOP, win_length=K_FFTSIZE,  center=False))
# print('esc50Wav2Spect" magspec max is ' + str(np.max(S)) +  ', and magspec sum is ' + str(np.sum(S)) + ', and magspec min is ' + str(np.min(S)))

# Sfoo = np.delete(S, (0), axis=0)  # delete freq 0 row
# print('esc50Wav2Spect" Sfoo max is ' + str(np.max(Sfoo)) +  ', and Sfoo sum is ' + str(np.sum(Sfoo)) + ', and Sfoo min is ' + str(np.min(Sfoo)))


# #D = librosa.logamplitude(S**2, ref_power=np.max)
# # computes amplitude_to_dB (which is in the man, but not in the library for some reason)
# D = librosa.amplitude_to_db(S, ref=np.max)
# print('spectrogram D shape is ' + str(D.shape))
# print('esc50Wav2Spect" ampDB max is ' + str(np.max(D)) +  ', and ampDB min is ' + str(np.min(D)))


# plt.subplot(3, 1, 1)
# librosa.display.specshow(D, y_axis='linear', x_axis='time', sr=K_SR, hop_length=K_HOP)
# plt.colorbar(format='%+2.0f dB')
# plt.title('Linear-frequency magnitude spectrogram')

# # save spectrogram somewhere
# tiffspect.logSpect2Tiff(D, 'foo.tif')

# # ====================
# # Proof that spectrogram writer/reader is working properly

# D1 = tiffspect.Tiff2LogSpect('foo.tif')
# plt.subplot(3, 1, 3)
# librosa.display.specshow(D, y_axis='linear', x_axis='time', sr=K_SR, hop_length=K_HOP)
# plt.colorbar(format='%+2.0f dB')
# plt.title('From Tiff: Linear-frequency magnitude spectrogram')
# tiffspect.logSpect2Tiff(D, 'foo.tif')